In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
from tradingview_ta import TA_Handler, Interval, Exchange
import tradingview_ta
from tradingview_ta import *
print(tradingview_ta.__version__)

3.3.0


In [3]:
sasa = TA_Handler(
    symbol="SASA",
    screener="turkey",
    exchange="BIST",
    interval=Interval.INTERVAL_1_MINUTE
)

print(sasa.get_analysis().summary)

{'RECOMMENDATION': 'BUY', 'BUY': 14, 'SELL': 3, 'NEUTRAL': 9}


In [17]:
df = pd.read_csv('./bist_symbol_data/ana_market.csv', index_col='Order')
bist_symbols = df['Code'].to_list()
bist_symbols = [f'bist:{i.lower()}' for i in bist_symbols]

# for initial position -1, then we chack if condition satisfies convert 1, if not 0
bist_symbols_dict = [(i.upper(),-1) for i in bist_symbols] 
bist_symbols_dict = dict(bist_symbols_dict)
# return 'bist:sasa':-1

In [5]:
analysis = sasa.get_analysis()
ind = analysis.indicators

In [6]:
analysis = get_multiple_analysis(
        screener="turkey", 
        interval=Interval.INTERVAL_1_HOUR, 
        symbols=bist_symbols
        )

In [12]:
# EMA5 >= SMA10, MACD.macd > MACD.signal, Ichimoku.BLine < close, ADX+DI >= ADX-DI, SMA10 < close, Stoch.K > Stoch.D 
def indicator_strategy_1(stock_ind: dict, symbol: str):

    #current_symbol = TA_Handler(symbol= symbol.split(':')[1], screener="turkey", exchange= symbol.split(':')[1],interval=Interval.INTERVAL_1_MINUTE        )
    #current_price = current_symbol.get_analysis().indicators['close']
    
    pattern1 = stock_ind['EMA5'] >= stock_ind['SMA10']
    pattern2 = stock_ind['MACD.macd'] > stock_ind['MACD.signal']
    pattern3 = stock_ind['Ichimoku.BLine'] < stock_ind['close']
    pattern4 = stock_ind['ADX+DI'] >= stock_ind['ADX-DI']
    pattern5 = stock_ind['SMA10'] < stock_ind['close']
    pattern6 = stock_ind['Stoch.K'] > stock_ind['Stoch.D']
    if(pattern1 and pattern2 and pattern3 and pattern4 and pattern5 and pattern6):
        print(f'Buy the stock {symbol}')

In [19]:
def indicator_strategy_buy_signal(stock_ind: dict, symbol_pair: dict, symbol: str):

    pattern1 = stock_ind['EMA5'] >= stock_ind['SMA10']
    pattern2 = stock_ind['MACD.macd'] > stock_ind['MACD.signal']
    pattern3 = stock_ind['Ichimoku.BLine'] < stock_ind['close']
    pattern4 = stock_ind['ADX+DI'] >= stock_ind['ADX-DI']
    pattern5 = stock_ind['SMA10'] < stock_ind['close']
    pattern6 = stock_ind['Stoch.K'] > stock_ind['Stoch.D']

    all_pattern = (pattern1 and pattern2 and pattern3 and pattern4 and pattern5 and pattern6)

    if all_pattern and symbol_pair[symbol] == 0:
        print(f'BUY SIGNAL FOR {symbol} at {stock_ind["close"]}')
        symbol_pair[symbol] = 1
    elif all_pattern and symbol_pair[symbol] == -1:
        symbol_pair[symbol] = 1
    elif not all_pattern and symbol_pair[symbol] == 1:
        print(f'SELL SIGNAL FOR {symbol} at {stock_ind["close"]}')
        symbol_pair[symbol] = 0
    else:
        symbol_pair[symbol] = 0

In [20]:
import time
while True:
    analysis = get_multiple_analysis(
        screener="turkey", 
        interval=Interval.INTERVAL_15_MINUTES, 
        symbols=bist_symbols
        )
    for k in analysis:
        dict_ind = analysis[k].indicators
        indicator_strategy_buy_signal(dict_ind, bist_symbols_dict, k)
    time.sleep(60*10)

KeyboardInterrupt: 

In [ ]:
for k in analysis:
    dict_ind = analysis[k].indicators
    indicator_strategy_1(dict_ind, k)

In [19]:
import datetime 
import pandas as pd

from dotenv import load_dotenv

load_dotenv()

api_id=os.getenv('TG_api_id')
api_hash = os.getenv('TG_hash_id')
phone = os.getenv('TG_phone')
username = os.getenv('TG_username')

In [ ]:
from telethon.sync import TelegramClient

client = TelegramClient(None, api_id, api_hash)

async def main():
    chats = ['...']
    chat = '...'
    df = pd.DataFrame()
    async for message in client.iter_messages(chat, limit=1000):
        data = {"group": chat, "sender": message.sender_id, "text": message.text, "date": message.date}
        temp_df = pd.DataFrame(data)
        df = df.append(temp_df)

    df['date'] = df['date'].dt.tz_localize(None)  ## supheli tekrara bak
    print(df.head())
    df.to_csv('./burda.csv')

async with client:
    client.loop.run_until_complete(main())

In [18]:
import nest_asyncio
nest_asyncio.apply()